# WCD SSD matrix signal model  

<br>
\begin{align}
\begin{pmatrix}
 S_S \\
 S_W
 \end{pmatrix} = 
 \begin{pmatrix}
 A_S(\chi) L_S{(\chi)} & 0 \\
  0 & A_W(\chi)L_W(\chi)
 \end{pmatrix}
 \begin{pmatrix}
1 & \ 1 \\[5pt]
\alpha & \beta
 \end{pmatrix}
 \begin{pmatrix}
 \rho^{\rm em} \\
 \rho^\mu
 \end{pmatrix}.
\end{align}

$\rho$ is the particle density.

$L_{\rm det}(\chi)$ is the track length in units of VEM or MIP such that $L_{\rm det}(0) = 1$. The $\chi$ dependence in the first matrix cancels on average suc that $A_{\rm det}(\chi) L_{\rm det}(\chi) =  A_{\rm det}(0)$.  
Then 
\begin{align}
    \alpha = \mathcal{G} \frac{S_W^{em}}{S_S^{em}} \\
    \beta = \mathcal{G} \frac{S_W^{\mu}}{S_S^{\mu}}
\end{align}
and $\mathcal{G} \approx 0.378\ \mathrm{m^2\, VEM/MIP}$.

## Signal model

The electromagnetic part is the sum of the convolution of the energy spectrum and the detector response for photon and electrons/positrons (e):

\begin{align}
    S_W^{em} &=  A_W(\chi)\left[ \int dE \frac{d\rho^e}{dE} \mathcal{R}_{\rm WCD}^e(E, \chi) + \int dE \frac{d\rho^\gamma}{dE} \mathcal{R}_{\rm WCD}^\gamma(E, \chi) \right] \\
    S_S^{em} &= A_S(\chi) \left[ \int dE \frac{d\rho^e}{dE} \mathcal{R}_{\rm SSD}^e(E, \chi) + \int dE \frac{d\rho^\gamma}{dE}\mathcal{R}_{\rm SSD}^\gamma(E, \chi) \right]
\end{align}

The corresponding muon part:
\begin{align}
    S_W^{\mu} &=  A_W(\chi)\int dE \frac{d\rho^\mu}{dE} \mathcal{R}_{\rm WCD}^\mu(E, \chi) \\
    S_S^{\mu} &=  A_S(\chi)\int dE \frac{d\rho^\mu}{dE} \mathcal{R}_{\rm SSD}^\mu(E, \chi) \\
\end{align}

## The detector response  
The average detector response is studied with simulations of single particles through the Auger station

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tables as tb
from load_detector_response import load_detector_data
from time_templates.utilities.plot import plot_profile_1d
from scipy.optimize import curve_fit
import uproot
from uncertainties import unumpy, ufloat
import json
plt.rcParams.update({"text.usetex": True, "font.size": 20, "font.family":"sans"})
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
markers = ['s', 'o', '^', 'v', 'x', '*']
lss = ['-', '--', ':', '-.']

PMT1Charge_to_VEM = 1595.45
PMT1Peak_to_VEM = 215.3
PMT2Charge_to_VEM = 1575.99
PMT2Peak_to_VEM = 214.3

PMT3Charge_to_VEM = 1586.53
PMT3Peak_to_VEM = 216.76
PMT5Charge_to_VEM = 175#196.89
PMT5Peak_to_VEM = 59.8

### Signal vs energy of particles

In [ ]:
# This data is from SingleParticleInjection custom offline module, that injects particles into the tank

# these particles go vertically through the SSD and WCD at xyz: 0.986 -0.355 2.0

datadir = '/home/mart/auger/data/SingleParticleVEM/'

muon_vert = load_detector_data(datadir + "UUB_SSD_muon_vert_flatlgE.root")
electron_vert = load_detector_data(datadir + "UUB_SSD_electron_vert_flatlgE.root")
photon_vert = load_detector_data(datadir + "UUB_SSD_photon_vert_flatlgE.root")
proton_vert = load_detector_data(datadir + "UUB_SSD_protons_vert_flatlgE.root")

In [ ]:
muon_vert_WCD = load_detector_data(datadir + "UUB_muon_vert_flatlgE.root")
electron_vert_WCD = load_detector_data(datadir + "UUB_electron_vert_flatlgE.root")
photon_vert_WCD = load_detector_data(datadir + "UUB_photon_vert_flatlgE.root")
proton_vert_WCD = load_detector_data(datadir + "UUB_protons_vert_flatlgE.root")

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7), sharey=True, sharex=True)
def brk_pwl(x, A, alpha, beta, gamma, c1, c2):
    S = np.where(x < c2,
                 np.where(x < c1, x**alpha, c1**(alpha-beta)*x**beta),
                 c1**(alpha-beta)*c2**(beta-gamma)*x**gamma)
    return A*S

energy_response_parameters = {}

def plot_comp_response_vs_E(df, color, ax, key='VEMCharge', fit=True, p0=None,  bounds=None, 
                            scatter=True, **kwargs):
    df[key] = np.where(df[key] < 0, 0, df[key])
    if scatter:
        ax.scatter(np.log10(df['kinEnergy']), df[key], marker='.',
                   alpha=0.03, color=color)
    ax, (x, y, yerr) = plot_profile_1d(np.log10(df['kinEnergy']), df[key], bins=np.linspace(6, 10, 30), ax=ax, stat='mean', 
                    ms=7, marker='s', color=color, **kwargs)
    
    if fit:
        mask = np.isfinite(y) & (yerr > 0) & (y > 1e-5) & (y < 20)
        if p0 is None:
            p0= [0.01, 1., -1., 0., 3., 10.]      
        if bounds is None:
            bounds=[(0, -3, -3, -3, 0, 0), (100, 5, 5, 5, 1e6, 1e6)]
        try:
            popt, pcov = curve_fit(brk_pwl, 10**x[mask]/1e6, y[mask], sigma=yerr[mask], p0=p0,
                              bounds=bounds, maxfev=2000)
        except:
            print("Fitting FAILED!!!!!!!!!!!")
            popt = p0
        
        print(popt)
        return list(popt)

    
# muon VEM with ssd
plot_comp_response_vs_E(muon_vert, 'b', ax1, mfc='none',
                                     p0=[0.0001, 2, 0.2, 0, 400, 2000],
                                     bounds=[(0, 0, 0, -0.1, 10, 500), (1e4, 4, 1, 0.1, 1e4, 1e4)])
#muon vem wcd
popt_mu_wcd = plot_comp_response_vs_E(muon_vert_WCD, 'b', ax1,
                                     scatter=False, label='muons',
                                     p0=[0.0001, 2, 0.2, 0, 400, 2000],
                                     bounds=[(0, 0, 0, -0.1, 10, 500), (1e4, 4, 1, 0.1, 1e4, 1e4)])
#muon ssd
popt_mu_ssd = plot_comp_response_vs_E(muon_vert, 'b', ax2, 'MIPCharge',
                                      p0=[0.1, 2, -2, 0, 10, 100],
                                      bounds=[(0, 0, -4, -0.01, 0, 0), (1e9, 5, 5, 0.01, 1e4, 1e4)])

#electron vem
plot_comp_response_vs_E(electron_vert, 'r', ax1, mfc='none',
                                     p0=[0.001, 1, 1, 1, 200, 2000],
                                     bounds=[(0, 0, 0, 0, 0, 0), (1e9, 5, 5, 5, 1e4, 1e4)])
#electron vem wcd, no ssd
popt_el_wcd = plot_comp_response_vs_E(electron_vert_WCD, 'r', ax1, label='electrons',
                                     scatter=False, p0=[0.001, 1, 1, 1, 200, 2000],
                                     bounds=[(0, 0, 0, 0, 0, 0), (1e9, 5, 5, 5, 1e4, 1e4)])
#electron ssd
popt_el_ssd = plot_comp_response_vs_E(electron_vert, 'r', ax2, 'MIPCharge')

#photon vem
plot_comp_response_vs_E(photon_vert, 'y', ax1, mfc='none',
                                     p0=[0.001, 1, 1, 1, 200, 2000],
                                     bounds=[(0, 0, 0, 0, 0, 0), (1e9, 5, 5, 5, 1e4, 1e4)])    
#photon vem wcd no ssd
popt_ph_wcd = plot_comp_response_vs_E(photon_vert_WCD, 'y', ax1, label='photons',
                                     scatter=False, p0=[0.001, 1, 1, 1, 200, 2000],
                                     bounds=[(0, 0, 0, 0, 0, 0), (1e9, 5, 5, 5, 1e4, 1e4)])
#photon ssd
popt_ph_ssd = plot_comp_response_vs_E(photon_vert, 'y', ax2, 'MIPCharge')


#protons wcd no ssd
plot_comp_response_vs_E(proton_vert_WCD, 'g', ax1, label='protons',
                        scatter=False,
                        p0=[0.001, 1, 1, 1, 200, 2000],
                        bounds=[(0, 0, 0, 0, 0, 0), (1e9, 5, 5, 5, 1e4, 1e4)]) 

#protonsvem
plot_comp_response_vs_E(proton_vert, 'g', ax1, mfc='none', 
                                     p0=[0.001, 1, 1, 1, 200, 2000],
                                     bounds=[(0, 0, 0, 0, 0, 0), (1e9, 5, 5, 5, 1e4, 1e4)])   
#protons mip
plot_comp_response_vs_E(proton_vert, 'g', ax2, 'MIPCharge',
                                     p0=[0.001, 1, 1, 1, 200, 2000],
                                     bounds=[(0, 0, 0, 0, 0, 0), (1e9, 5, 5, 5, 1e4, 1e4)])    

energy_response_parameters['muon_wcd'] = popt_mu_wcd
energy_response_parameters['muon_ssd'] = popt_mu_ssd
energy_response_parameters['electron_wcd'] = popt_el_wcd
energy_response_parameters['electron_ssd'] = popt_el_ssd
energy_response_parameters['photon_wcd'] = popt_ph_wcd
energy_response_parameters['photon_ssd'] = popt_ph_ssd

with open('../data/UUB_energy_response_parameters.json', 'w') as out:
    json.dump(energy_response_parameters, out, indent=4)


Rmuwcd = lambda x: brk_pwl(10**x/1e6, *popt_mu_wcd)
Rmussd = lambda x: brk_pwl(10**x/1e6, *popt_mu_ssd)
Relwcd = lambda x: brk_pwl(10**x/1e6, *popt_el_wcd)
Relssd = lambda x: brk_pwl(10**x/1e6, *popt_el_ssd)
Rphwcd = lambda x: brk_pwl(10**x/1e6, *popt_ph_wcd)
Rphssd = lambda x: brk_pwl(10**x/1e6, *popt_ph_ssd)
x = np.linspace(5, 11, 100)

# ax1.plot(x, Rmuwcd(x), 'b--', lw=3)
# ax2.plot(x, Rmussd(x), 'b:', lw=3)

# ax2.plot(x, Relssd(x), 'r:', lw=2)
# ax1.plot(x, Relwcd(x), 'r--', lw=2)

# ax1.plot(x, Rphwcd(x), 'y--', lw=2)
# ax2.plot(x, Rphssd(x), 'y:', lw=2)


ax1.plot([0], [0], mfc='none', marker='s', ls='', color='k', label='also through SSD')
lgEvem = np.log10(400e6)
ax1.plot(x, np.where(x < lgEvem, (10**x/10**lgEvem)**2, 1), 'b-')
ax1.plot(x, 10**x/10**lgEvem, 'r-')
ax2.plot(x, np.ones_like(x), 'b-')
ax2.plot(x, np.ones_like(x), 'r--')


# ax1.set_xscale('log')
ax1.set_yscale('log')
ax2.yaxis.set_tick_params(labelleft=True)
ax1.set_ylabel('$\\langle S_{\\rm WCD} \\rangle$ [VEM] ')
ax2.set_ylabel('$\\langle S_{\\rm SSD} \\rangle$ [MIP]')
ax1.set_title('WCD')
ax2.set_title('SSD')
ax1.set_xlabel('$\\log_{10}\\left(E_{\\rm kin}/\\mathrm{eV}\\right)$')
ax2.set_xlabel('$\\log_{10}\\left(E_{\\rm kin}/\\mathrm{eV}\\right)$')
ax1.legend(fontsize=16)
plt.tight_layout()
ax1.set_xlim([6, 10])
ax1.set_ylim([0.001, 20])
plt.savefig('signal_response_vs_kin_energy.pdf', bbox_inches='tight')

In [ ]:
print(f"Evem = {popt_mu_wcd[-2]:.0f} MeV, kinetic energy")


In the above plot it is shown that the response of the WCD to electrons+photons is approximately linear $ S \sim E/E_{VEM}$, with $E_{VEM} \approx 400$ MeV. The muon response of the WCD is flat above $E_{VEM}$ and below that goes like $S \sim E^{-2}$.  
The SSD response is approximately flat in energy, except for the bump for low energy muons, but we will ignore this since later we will show that there are hardly any muons below 100 MeV. The response of the SSD to photons is almost zero, except for a tiny fraction of photons that convert in the aluminium, which will be important later.  
High energy photons and electrons (above 1 GeV) or so start to deviate from linearity because some signal is leaked out of the tank. At low energy, some electrons and photons do not make it to the tank. 

### Signal versus the track length (incoming direction)

In [ ]:
# The particle positions are sampled in a disk above the detector and this disk is rotated such that one gets 
# all possible track lengths in the dete|ctor

muon_2GeV = load_detector_data(datadir + "UUB_SSD_muon_disk_2GeV_widerdisk.root")
electron_100MeV = load_detector_data(datadir + "UUB_SSD_electron_disk_100MeV.root")
photon_100MeV = load_detector_data(datadir + "UUB_SSD_photon_disk_100MeV.root")

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7), sharey=True, sharex=True)

def Lwcd(theta):
    return 1/(np.cos(theta) + 0.42*np.sin(theta))

def Lssd(theta):
    return 1/np.cos(theta)


sct_bins = np.linspace(1, 2, 20) #np.array([1, 1.05, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.8, 1.9, 2])
def plot_comp_response_vs_cos_theta(df, color, ax, key='VEMCharge', plot_scatter=False, **kwargs):
    
    if plot_scatter:
        cutstr = f'{key} > 0'
        df = df.query(cutstr)
        ax.scatter(1/df['cosTheta'], df[key], marker='.',
                   alpha=0.025, color=color)

    plot_profile_1d(1/df['cosTheta'], df[key], ax=ax, bins=sct_bins, stat='mean', 
                     marker='s', color=color, **kwargs)
        
plot_comp_response_vs_cos_theta(muon_2GeV, 'b', ax1, remove_outliers=(0.0, 10))
plot_comp_response_vs_cos_theta(electron_100MeV, 'r', ax1)
plot_comp_response_vs_cos_theta(photon_100MeV, 'y', ax1)

plot_comp_response_vs_cos_theta(muon_2GeV, 'b', ax2, key='MIPCharge',
                                remove_outliers=(0.1, 10))
plot_comp_response_vs_cos_theta(electron_100MeV, 'r', ax2, key='MIPCharge', remove_outliers=(0.1, 10))
# plot_comp_response_vs_cos_theta(photon_100MeV, 'y', ax2, key='MIPCharge')

theta = np.linspace(0, np.pi/3)
ax1.plot(1/np.cos(theta), Lwcd(theta), 'k--', label='$L_{\\rm WCD}(\chi)$', lw=2, zorder=99)
ax2.plot(1/np.cos(theta), Lssd(theta), 'k:', label='$L_{\\rm SSD}(\chi)$', lw=2, zorder=99)
ax1.legend()
ax2.legend()
ax2.yaxis.set_tick_params(labelleft=True)
ax1.set_ylabel('$\\langle S_{\\rm WCD} \\rangle$ [VEM] ')
ax2.set_ylabel('$\\langle S_{\\rm SSD} \\rangle$ [MIP]')
ax1.set_title('WCD')
ax2.set_title('SSD')
ax1.set_xlabel('$\sec{\\chi}$')
ax2.set_xlabel('$\sec{\\chi}$')
ax1.set_ylim([0., 2.5])
plt.tight_layout()
plt.savefig('signal_response_vs_sec_theta.pdf', bbox_inches='tight')

One can appriciate in the plot above that in the WCD only the muon signal depends on the track-length. The electrons and photons here have an energy of 100 MeV and are thus completly stopped in the tank (for average track-lenghts). Since there are not many electrons and photons above 1 GeV we will ignore the track lenght for the elektromagnetic part in the WCD. The same argument holds for muons below $E_{VEM}$: their signal also does not depend on the track-length.

The theoretical curve for the mean track-length in the WCD slightly underestimates the average signal vs sec(chi). Note that the spread increase very rapidly and the mode does not correspond to the mean.  

In the SSD electrons and muons follow nicely the 1/cos(chi) dependence as expected for an infinite thin plane. Photons are not shown here since they mostly do not make a signal.


In [ ]:
def calc_eff(df, det='WCD', Elow=1e5, Ehigh=1e12, cut=0.001):
    df = df.query('(kinEnergy > @Elow) & (kinEnergy < @Ehigh)')
    n = len(df)
    if n == 0:
        return np.nan
    if det == 'WCD':
        key = "VEMCharge"
    else:
        key = "MIPCharge"
    m = len(df.query(f'{key} > @cut'))
    return m/n

def calc_eff_Ecuts(df, Elows, det='WCD', cut=0.001):
    out = np.zeros_like(Elows)
    for i, Ecut in enumerate(Elows):
        out[i] = calc_eff(df, det=det, Elow=Ecut, cut=cut)
    return out

def calc_eff_Ebins(df, Ebins, det='WCD', cut=0.001):
    n = len(Ebins) - 1
    out = np.zeros(n)
    for i in range(n):
        out[i] = calc_eff(df, det=det, Elow=Ebins[i], Ehigh=Ebins[i+1], cut=cut)
    return out

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7), sharey=True, sharex=True)

Ebins = np.logspace(6, 10, 30)
mE = (Ebins[1:] + Ebins[:-1])/2
mlgE = (np.log10(Ebins[:-1]) + np.log10(Ebins[1:]))/2
markers = ['o', 'x', '^', 'v']

ADCcuts = [1, 3]#2, 4]

from scipy.optimize import curve_fit
from scipy import special

def func_erf(x, a, b, mu, sigma):
    return a + (b-a)*0.5*(1+special.erf((x-mu)/sigma))

    return c/(d + a*np.exp(-b*x))

def fit_func(func, x, y, ax, **kwargs):
    mask = np.isfinite(y)
    popt, pcov = curve_fit(func, x[mask], y[mask], p0=[0, 1, 7.5, 1],
                          maxfev=10000)
    x = np.logspace(x.min(), x.max(), 100)
    ax.plot(x, func(np.log10(x), *popt), **kwargs)
    return popt

WCDCharge_to_VEM = 1585.99
SSDCharge_to_VEM = 59.8

lss = ['-', '--', ':']


for ADCcut, m, ls in zip(ADCcuts, markers, lss):
    
    VEMcut = ADCcut / WCDCharge_to_VEM
    MIPcut = ADCcut / SSDCharge_to_VEM
    w_mu_eff = calc_eff_Ebins(muon_vert_WCD, Ebins, 'WCD', VEMcut)
    popt = fit_func(func_erf, mlgE, w_mu_eff, ax1, color='b', ls=ls)

    ax1.plot(mE, w_mu_eff, color='b', marker=m, ls='')
    f_wcd_mu_eff = lambda x: func_erf(x, *popt)
    print("Muon eff wcd", f_wcd_mu_eff(np.log10(300e6)), f_wcd_mu_eff(9))
    
    w_el_eff = calc_eff_Ebins(electron_vert_WCD, Ebins, 'WCD', VEMcut)
    popt = fit_func(func_erf, mlgE, w_el_eff, ax1, color='r', ls=ls)
    ax1.plot(mE, w_el_eff, color='r', marker=m, ls='')

    f_wcd_el_eff = lambda x: func_erf(x, *popt)
    print("El eff wcd", f_wcd_el_eff(np.log10(25e6)))

    w_ph_eff = calc_eff_Ebins(photon_vert_WCD, Ebins, 'WCD', VEMcut)
    popt = fit_func(func_erf, mlgE, w_ph_eff, ax1, color='y', ls=ls)
    ax1.plot(mE, w_ph_eff, color='y', marker=m, ls='')
    f_wcd_ph_eff = lambda x: func_erf(x, *popt)
    print("Ph eff wcd", f_wcd_ph_eff(np.log10(7e6)))
    
    #SSD
    s_mu_eff = calc_eff_Ebins(muon_vert, Ebins, 'SSD', MIPcut)
    popt = fit_func(func_erf, mlgE, s_mu_eff, ax2, color='b', ls=ls)
    ax2.plot(mE, s_mu_eff, color='b', marker=m, ls='')
    f_ssd_mu_eff = lambda x: func_erf(x, *popt)
    print("mu eff ssd", f_wcd_mu_eff(9))
    
    s_el_eff = calc_eff_Ebins(electron_vert, Ebins, 'SSD', MIPcut)
    popt = fit_func(func_erf, mlgE, s_el_eff, ax2, color='r', ls=ls)
    ax2.plot(mE, s_el_eff, color='r', marker=m, ls='')
    f_ssd_el_eff = lambda x: func_erf(x, *popt)
    print("el eff ssd", f_ssd_el_eff(np.log10(25e6)))

    s_ph_eff = calc_eff_Ebins(photon_vert, Ebins, 'SSD', MIPcut)
    popt = fit_func(func_erf, mlgE, s_ph_eff, ax2, color='y', ls=ls)
    ax2.plot(mE, s_ph_eff, color='y', marker=m, ls='')
    f_ssd_ph_eff = lambda x: func_erf(x, *popt)
    print("ph eff ssd", f_ssd_ph_eff(np.log10(7e6)))
    
    ax1.plot([-1], [-1], marker=m, ls='', color='k', label='$S > %i$ fADC' % ADCcut)


ax1.legend()
ax2.yaxis.set_tick_params(labelleft=True)
ax1.set_ylabel('WCD efficiency')
ax2.set_ylabel('SSD efficiency')
ax1.set_title('WCD')
ax2.set_title('SSD')
ax1.set_xlabel('$E_{\\rm kin}$ [eV]')
ax2.set_xlabel('$E_{\\rm kin}$ [eV]')
# ax1.set_ylim([-0.1, 1.1])
ax1.set_xscale('log')
ax1.grid()
ax2.grid()
ax1.set_yscale('log')
ax2.set_yscale('log')
ax1.set_ylim([1e-2, 2])
ax2.set_ylim([1e-2, 2])
plt.tight_layout()
# plt.savefig('detector_efficiency_vs_E.pdf', bbox_inches='tight')

## The energy spectrum of particles

### The signal model now reads:

\begin{align}
    S_W^{\mu} &=  A_W(\chi)\int dE \frac{d\rho^\mu}{dE} \min{\left[ \left(\frac{E}{E_{VEM}}\right)^2, L_W(\chi) \right]} \\
    S_S^{\mu} &=  A_S(\chi)L_S(\chi) \rho^\mu \\
\end{align}


\begin{align}
    S_W^{em} &=  A_W(\chi)\left[ \epsilon^e_{W} \rho^e  \frac{\langle E^e \rangle }{E_{\rm VEM}} + \epsilon^\gamma_{W} \rho^\gamma \frac{\langle E^\gamma \rangle }{E_{\rm VEM}}\right] \\
    S_S^{em} &= A_S(\chi)L_S(\chi) \left[ \epsilon^e_{S} \rho^e  + \epsilon^\gamma_{S} \rho^\gamma \right]
\end{align}
So that:
\begin{align}
    \alpha &= \frac{A_S L_S}{A_W L_W} \frac{S_W^{em}}{S_S^{em}} = \frac{1}{L_W(\chi)} \left[ \epsilon^e_{W} \frac{\langle E^e \rangle }{E_{\rm VEM}} + \epsilon^\gamma_{W} r^{\gamma/e} \frac{\langle E^\gamma \rangle }{E_{\rm VEM}}\right] \bigg/ \left[ \epsilon^e_{S} + \epsilon^\gamma_{S} r^{\gamma/e} \right] \\
     \beta &= \frac{A_S L_S}{A_W L_W} \frac{S_W^{\mu}}{S_S^{\mu}} = \left[ \int dE \frac{d\rho^\mu}{dE} \min{\left[ \left(\frac{E}{E_{VEM}}\right)^2\frac{1}{L_W(\chi)}, 1 \right]} \right] \bigg / \rho^\mu \\
     &= \epsilon^\mu_{W, 1}\frac{\langle (E^\mu_{E < E_{VEM}})^2 \rangle}{E^2_{VEM}} f^\mu_{E < E_{VEM}} + (1-f^\mu_{E < E_{VEM}})
\end{align}
$\epsilon^\mu_{W, 1}$ is the efficiency of muons below $E_{VEM}$. Actually, 3 terms should appear in the last line, since one can also have muons below the Cherenkov threshold, that do not make any signal in the WCD. Making $\beta$ slightly smaller.  

It is clear that the values for $\alpha$ and $\beta$ depend on the energy spectrum


In [ ]:
def f_cos_chi(r, theta, psi, zmax=None):
    if zmax is None:
        zmax = 7400/np.cos(theta) * np.log(880/(500*np.cos(theta)))
    Delta = r*np.tan(theta)*np.cos(psi)
    return zmax * np.cos(theta)/np.sqrt(r**2 + (zmax-Delta)**2)

def f_chi(r, theta, psi, zmax=None):
    return np.arccos(f_cos_chi(r, theta, psi, zmax))

def f_Awcd(chi, R=1.8, H=1.2):
    return (np.pi*R**2*np.abs(np.cos(chi)) + 2*R*H*np.sin(chi))

def f_Assd(chi):
    return 3.84*np.abs(np.cos(chi))

def f_Lwcd(chi, R=1.8, H=1.2):
    return 1/(np.abs(np.cos(chi)) + 2*H/(np.pi*R)*np.sin(chi))

def f_Lssd(chi):
    return 1/np.cos(chi)

r = np.linspace(0, 2200)
theta = np.linspace(0, np.pi/4, 3)
psi = np.linspace(0, np.pi, 3)

f, axes = plt.subplots(3, 3, figsize=(13, 14), sharey=True, sharex=True)
Xmaxs = np.array([450, 750])

lss = ['-', '--', ':']

for i, th in enumerate(theta):
    ax = axes[0, i]
    ax1 = axes[1, i]
    ax2 = axes[2, i]
    for j, p in enumerate(psi):
        for k, X in enumerate(Xmaxs):
            z = 7400/np.cos(th) * np.log(880/(X*np.cos(th)))
            ax.plot(r, 1/f_cos_chi(r, th, p, z), 
                    color=colors[j], ls=lss[k])
            ax1.plot(r, f_Lwcd(f_chi(r, th, p, z)), color=colors[j], ls=lss[k])
            ax2.plot(r, f_Lssd(f_chi(r, th, p, z)), color=colors[j], ls=lss[k])
    ax.axhline(1/np.cos(th), ls='--', color='k')
    ax.set_title(f'$\\theta = {np.rad2deg(th):.0f}^\circ$')
    ax1.axhline(1, ls='-', color='k')
    ax2.axhline(1, ls='-', color='k')
    ax2.set_xlabel('$r$ [m]')

    
    axes[i, 0].plot(0, 0, 'C0-', label='$\\psi = 0^\\circ$')
    axes[i, 0].plot(0, 0, 'C1-', label='$\\psi = 90^\\circ$')
    axes[i, 0].plot(0, 0, 'C2-', label='$\\psi = 180^\\circ$')
    axes[i, 0].plot(0, 0, 'k-', label='$X = 450\, \\mathrm{g/cm^2}$')
    axes[i, 0].plot(0, 0, 'k--', label='$X = 750\, \\mathrm{g/cm^2}$')

    axes[i, 0].legend(ncol=1, fontsize=14, loc=2)

axes[0, 0].annotate('$\\chi = \\theta$', xy=(1800, 0.90))
axes[0, 0].set_ylabel('$\\sec{\\chi}$')
axes[1, 0].set_ylabel('$\\langle S_{\\rm WCD} \\rangle$ [VEM]')
axes[2, 0].set_ylabel('$\\langle S_{\\rm SSD} \\rangle$ [MIP]')

plt.subplots_adjust(wspace=0.0, hspace=0)
# plt.tight_layout()
ax.set_ylim([0.8, 2.2])
# plt.savefig('sec_chi_vs_r_theta_psi.pdf', bbox_inches='tight')

In [ ]:
#Loading energy spectra. These are histograms from corsika showers (Napoli, EPOS_LHC, corsika v?).
#Preprocessed to a hdf5 file for nice reading

# proton = tb.open_file('data/proton_Espec.h5')
proton = tb.open_file('data/proton_Espec_lgr.h5')
proton_aab = tb.open_file('data/proton_Aab.h5')
# iron = tb.open_file('data/iron_Espec2.h5')

In [ ]:
def Aground_wedge(R1, R2, psi1, psi2, theta=0):
    """
    The surface on the ground as projected from the shower plane
    R and psi are the distance and azimuth in the shower plane
    psi1 and psi2 should be between -pi, pi. and psi2 should be larger than psi1
    """
    
    AR = 0.5/np.cos(theta) * (R2**2 - R1**2) 
    
    psi_p1 = np.arctan2(np.sin(psi1)*np.cos(theta), np.cos(psi1))
    psi_p2 = np.arctan2(np.sin(psi2)*np.cos(theta), np.cos(psi2))
    
    AP2 = 0.25*( 2*psi_p2+np.sin(2*psi_p2)/np.cos(theta)**2 + 2*psi_p2 - np.sin(2*psi_p2))
    AP1 = 0.25*( 2*psi_p1+np.sin(2*psi_p1)/np.cos(theta)**2 + 2*psi_p1 - np.sin(2*psi_p1))
#     print(AP2, AP1)
    return AR * (AP2 - AP1)


f, ax = plt.subplots(1)

thetas = np.linspace(0, np.pi/2*0.8, 100)
for psis in [(-np.pi/4, np.pi/4), (np.pi/4, 3*np.pi/4)]:
    l = []
    for t in thetas:
        l.append(Aground_wedge(0, 1000, psis[0], psis[1], t))
    ax.plot(np.rad2deg(thetas), l,
            label=f'$\int_{{\\psi={np.rad2deg(psis[0]):.0f}^\circ}}^{{\\psi={np.rad2deg(psis[1]):.0f}^\circ}}$')
    ax.axhline(np.pi*1000**2, ls='--', color='k')

ax.legend()
ax.set_ylabel('Area [$\\mathrm{m^2}$]')
ax.set_xlabel('$\\theta$ [deg]')


In [ ]:
def get_bin(x, bins):
    ix = np.nanargmin(np.abs(bins - x))
    if ix >= len(bins) -1:
        ix = len(bins)-2
    return ix

from scipy.interpolate import RegularGridInterpolator


def get_spec(t_h5, ptype,
             lgE=(18.5, 20), cosTheta=(0.5, 1), Xmax=(0, 2000)):
    """ Returns 3d function (interpolated) dn/dlgEdA = f(r, psi, lgEkin)"""
    
    bins = t_h5.root.bins
    table = t_h5.root.event
    lgE_min, lgE_max = lgE
    cT_min, cT_max = cosTheta
    Xmax_min, Xmax_max = Xmax
    
    if ptype == 'muon':
        specs = t_h5.root.muon_spec
    elif ptype == 'electron':
        specs = t_h5.root.electron_spec
    elif ptype == 'photon':
        specs = t_h5.root.photon_spec

    cut = f'(Xmax >= {Xmax_min}) & (Xmax < {Xmax_max}) &\
            (cosTheta >= {cT_min}) & (cosTheta < {cT_max}) &\
            (lgE >= {lgE_min}) & (lgE < {lgE_max})'

    spec = np.array([specs[t['it']] for t in table.where(cut)])
#     spec = specs[1489]
#     print(table[1489])

#     print(spec.shape)
    if spec.shape[0] == 0:
        raise RuntimeError("No showers found")
    
    lgr_bins = bins.r_bins.read()
    lgEkin_bins = bins.lgE_pkin_bins.read()
    psi_bins = bins.psi_bins.read()
    
    lgrmean = (lgr_bins[1:] + lgr_bins[:-1])/2
    r_bins = 10**lgr_bins
    rmean = 10**lgrmean
    lgEkin = (lgEkin_bins[1:] + lgEkin_bins[:-1])/2
    dlgEkin = (lgEkin_bins[1:] - lgEkin_bins[:-1])

    E = 10**lgEkin

    psi_mean = (psi_bins[1:] + psi_bins[:-1])/2
    psi_mean[0] = -np.pi
    psi_mean[-1] = np.pi

    #####################
    ## The area on the ground is an ellipse with major axis r/cos(theta) and minor just r.
    ## Not sure if the psi in the shower plane should change as wel
    ## A_ellipse = pi r^2/cos(theta)
    #####################
    
    ######
    ## ' = on the ground, while psi and r are in shower plane
    ## psi' = atan(tan(psi)cos(theta))
    ## r' = r*sqrt[cos^2(psi)/ cos^2(theta) + sin^2(psi)]
    ## dpsi'/dpsi = cos(theta)/[cos^2(psi + sin^2(psi)*cos^2(theta))]
    ## dr'/dr = sqrt[cos^2(psi)/ cos^2(theta) + sin^2(psi)]
    ## A'ij = int_R' int_psi' r' dr' dpsi' = int_R' int_psi' r/cos(theta) dr dpsi (everything cancels)
    ## A'ij = 0.5R^2/cos(theta) [cos^2(psi)/cos^2(theta) + sin^2(psi)] * arctan(tan(psi)*cos(theta))
#     print(spec.shape)

    theta = np.arccos((cT_min + cT_max)/2)
#     A = Aground_wedge(r_bins[1:][:, None], r_bins[:-1][:, None], 
#                       psi_bins[1:][None, :], psi_bins[:-1][None, :], theta)
    
    A = (np.pi*(r_bins[1:]**2 - r_bins[:-1]**2)/len(psi_mean)/np.cos(theta))[:, None]
    
    if len(spec.shape) < 4:
        spec_mean = spec#/A[:, :, None] / dlgEkin[None, None, :]
        spec_err = np.zeros_like(spec_mean)
    else:
        spec_mean = np.mean(spec, axis=0)/A[:, :, None] / dlgEkin[None, None, :]
        spec_err = spec.std(axis=0)/A[:, :, None] / dlgEkin[None, None, :]
        
    fin = RegularGridInterpolator((lgrmean, psi_mean, lgEkin),
                                  spec_mean,
                                  bounds_error=False,
                                  fill_value=0)
    fin_err = RegularGridInterpolator((lgrmean, psi_mean, lgEkin),
                                      spec_err,
                                      bounds_error=False,
                                      fill_value=0)
                                          
    fin_ = lambda r, psi, lgE: fin((np.log10(r), psi, lgE))
    fin_err_ = lambda r, psi, lgE: fin_err((np.log10(r), psi, lgE))
    return fin_, fin_err_

def get_E(t_h5):     
    bins = t_h5.root.bins
    lgEk = bins.lgE_pkin_bins.read()
    Ek = 10**lgEk
    mlgE = (lgEk[1:] + lgEk[:-1])/2
    dlgE = lgEk[1:] - lgEk[:-1]
    mE = (Ek[1:] + Ek[:-1])/2
    dE = Ek[1:] - Ek[:-1]
    return mlgE, dlgE, mE, dE

def integrate_spec(f, low=4, high=15, step=None, args=()):
    if step is None:
        step = (high - low)/10000
    x = np.arange(low, high, step)
    return f(x, *args).sum()*step


In [ ]:
def calc_average_wcd_em_signal_per_particle(spec_el, spec_ph):

    Ee = integrate_spec(lambda x: spec_el(x)*Relwcd(x))
    Ne = integrate_spec(lambda x: spec_el(x))
    
    Eph = integrate_spec(lambda x: spec_ph(x)*Rphwcd(x))
    Nph = integrate_spec(lambda x: spec_ph(x))
    
    return (Ee + Eph)/(Ne+Nph)

def calc_average_ssd_em_signal_per_particle(spec_el, spec_ph):

    Ee = integrate_spec(lambda x: spec_el(x)*Relssd(x))
    Ne = integrate_spec(lambda x: spec_el(x))
    
    Eph = integrate_spec(lambda x: spec_ph(x)*Rphssd(x))
    Nph = integrate_spec(lambda x: spec_ph(x))
    
    return (Ee + Eph)/(Ne+Nph)

def calc_average_wcd_mu_signal_per_particle(spec_mu):

    Emu = integrate_spec(lambda x: spec_mu(x)*Rmuwcd(x))
    Nmu = integrate_spec(lambda x: spec_mu(x))
    
    return Emu/Nmu

def calc_average_ssd_mu_signal_per_particle(spec_mu):

    Emu = integrate_spec(lambda x: spec_mu(x)*Rmussd(x))
    Nmu = integrate_spec(lambda x: spec_mu(x))
    
    return Emu/Nmu

from collections import defaultdict
from scipy.interpolate import interp1d
dd = defaultdict(list)

muon_specs = proton_aab.root.muon_spec
el_specs = proton_aab.root.electron_spec
ph_specs = proton_aab.root.photon_spec

bins = proton_aab.root.bins

lgr_bins = bins.r_bins.read()
lgEkin_bins = bins.lgE_pkin_bins.read()
psi_bins = bins.psi_bins.read()
lgEmean = (lgEkin_bins[1:] + lgEkin_bins[:-1])/2
lgrmean = (lgr_bins[1:] + lgr_bins[:-1])/2
dlgE = lgEkin_bins[1] - lgEkin_bins[0]
n = len(proton_aab.root.event)
for i, row in enumerate(proton_aab.root.event):
    print(f'\r {i+1}/{n}', end='')
    lgE = row['lgE']
    Xmax = row['Xmax']
    cosTheta = row['cosTheta']
    i = row['it']
    #sum psi
    mu_spec = muon_specs[i].sum(axis=1)
    el_spec = el_specs[i].sum(axis=1)
    ph_spec = ph_specs[i].sum(axis=1)
    
    for i, lgr in enumerate(lgrmean):
        mu_spec_r = interp1d(lgEmean, mu_spec[i], fill_value=0, bounds_error=False)
        el_spec_r = interp1d(lgEmean, el_spec[i], fill_value=0, bounds_error=False)
        ph_spec_r = interp1d(lgEmean, ph_spec[i], fill_value=0, bounds_error=False)
        
        b = calc_average_wcd_mu_signal_per_particle(mu_spec_r)
        d = calc_average_ssd_mu_signal_per_particle(mu_spec_r)
        a = calc_average_wcd_em_signal_per_particle(el_spec_r, ph_spec_r)
        c = calc_average_ssd_em_signal_per_particle(el_spec_r, ph_spec_r)
        dd['lgE'].append(lgE)
        dd['cosTheta'].append(cosTheta)
        dd['Xmax'].append(Xmax)
        dd['lgr'].append(lgr)
        dd['a'].append(a)
        dd['b'].append(b)
        dd['c'].append(c)
        dd['d'].append(d)
import pandas as pd

df = pd.DataFrame(dd)     

In [ ]:
df_gb = df.groupby(['lgE', 'cosTheta', 'lgr']).mean().reset_index()
df_gb.head()

In [ ]:
import seaborn as sns
plt.rcParams.update({"text.usetex": True, "font.size": 16, "font.family":"sans"})

f, axes = plt.subplots(2, 2, figsize=(15, 15))
axes = axes.flatten()
palette = ['r', 'g', 'b', 'm', 'c']
sns.scatterplot(x='lgr', y='a', hue='cosTheta', size='lgE', data=df_gb, ax=axes[0], palette=palette)
axes[0].legend(loc=2)
sns.scatterplot(x='lgr', y='b', hue='cosTheta', size='lgE', data=df_gb, ax=axes[1], legend=False,
                palette=palette)
sns.scatterplot(x='lgr', y='c', hue='cosTheta', size='lgE', data=df_gb, ax=axes[2], legend=False,
                palette=palette)
sns.scatterplot(x='lgr', y='d', hue='cosTheta', size='lgE', data=df_gb, ax=axes[3], legend=False,
                palette=palette)

axes[0].set_ylabel('wcd em signal / particle')
axes[1].set_ylabel('wcd muon signal / particle')
axes[2].set_ylabel('ssd em signal / particle')
axes[3].set_ylabel('ssd muon signal / particle')

In [ ]:
df_gb['alpha'] = 0.378*df_gb['c']/df_gb['a']
df_gb['beta'] = 0.378 * df_gb['d']/df_gb['b']


f, axes = plt.subplots(1, 2, figsize=(15, 7))
axes = axes.flatten()
sns.scatterplot(x='lgr', y='alpha', hue='cosTheta', size='lgE', data=df_gb, ax=axes[0], palette=palette)
axes[0].legend(loc=3)
sns.scatterplot(x='lgr', y='beta', hue='cosTheta', size='lgE', data=df_gb, ax=axes[1], legend=False,
                palette=palette)


In [ ]:
def integrate_spec_err(fin, ferr=None, g=lambda x: 1, norm=False, 
                       low=4, high=15, *args):
    f1 = lambda x: g(x)*fin(x)

    if norm:
        N1 = integrate_spec(fin, low, high, *args)
    else:
        N1 = 1
    int1 = integrate_spec(f1, low, high, *args)/N1

    if ferr is not None:

        fhigh = lambda x: fin(x) +  ferr(x)
        flow = lambda x: fin(x) - ferr(x)
        if norm:
            Nh = integrate_spec(fhigh, low, high, *args)
            Nl = integrate_spec(flow, low, high, *args)
        else:
            Nh = 1
            Nl = 1
        
        int_high = integrate_spec(lambda x: g(x)*fhigh(x), low, high, *args)/Nh
        int_low = integrate_spec(lambda x: g(x)*flow(x), low, high, *args)/Nl
        int_err = np.mean(np.abs([int1 - int_high, int1-int_low]))
        return ufloat(int1, int_err)
    else:
        return int1

def calc_Swcdmu_from_spec(spec_mu, r, psi, spec_mu_err=None, chi=0):
    
    #Awcd(chi)*Lwcd(chi) cancels (except for low energy...)
    Awcd = f_Awcd(chi)
    Lwcd = f_Lwcd(chi)
    
    spec_mu_rpsi = lambda x: spec_mu(r, psi, x)
    if spec_mu_err is not None:
        spec_mu_err_rpsi = lambda x: spec_mu_err(r, psi, x)
    else:
        spec_mu_err_rpsi = None
        
    #The LWCD should not hold for E <EVEM ! TODO: FIX: 
    return Awcd * Lwcd * integrate_spec_err(spec_mu_rpsi, spec_mu_err_rpsi,
                                     g=Rmuwcd)

def calc_Sssdmu_from_spec(spec_mu, r, psi, spec_mu_err=None, chi=0):
    Assd = f_Assd(chi)
    Lssd = f_Lssd(chi)
    spec_mu_rpsi = lambda x: spec_mu(r, psi, x)
    if spec_mu_err is not None:
        spec_mu_err_rpsi = lambda x: spec_mu_err(r, psi, x)
                                            
    else:
        spec_mu_err_rpsi = None
    return Assd * Lssd * integrate_spec_err(spec_mu_rpsi, spec_mu_err_rpsi,
                                    g=Rmussd)

def calc_Swcdel_from_spec(spec_el, r, psi, spec_el_err=None, chi=0):
    Awcd = f_Awcd(chi)
    spec_el_rpsi = lambda x: spec_el(r, psi, x)
    if spec_el_err is not None:
        spec_el_rpsi_err = lambda x: spec_el_err(r, psi, x)
    else:
        spec_el_rpsi_err = None
    return Awcd * integrate_spec_err(spec_el_rpsi, spec_el_rpsi_err, g=Relwcd)

def calc_Swcdph_from_spec(spec_ph, r, psi, spec_ph_err=None, chi=0):
    Awcd = f_Awcd(chi)
    spec_ph_rpsi = lambda x: spec_ph(r, psi, x)
    if spec_ph_err is not None:
        spec_ph_rpsi_err = lambda x: spec_ph_err(r, psi, x)
    else:
        spec_ph_rpsi_err = None
    return Awcd * integrate_spec_err(spec_ph_rpsi, spec_ph_rpsi_err, g=Rphwcd)


def calc_Swcdem_from_spec(spec_el, spec_ph, r, psi,
                          spec_el_err=None, spec_ph_err=None, chi=0):
    return ( calc_Swcdel_from_spec(spec_el, r, psi, spec_el_err, chi) + 
            calc_Swcdph_from_spec(spec_ph, r, psi, spec_ph_err, chi) )


def calc_Sssdel_from_spec(spec_el, r, psi, spec_el_err=None, chi=0):
    Assd = f_Assd(chi)
    Lssd = f_Lssd(chi)
    spec_el_rpsi = lambda x: spec_el(r, psi, x)
    if spec_el_err is not None:
        spec_el_rpsi_err = lambda x: spec_el_err(r, psi, x)
    else:
        spec_el_rpsi_err = None
    return Assd * Lssd * integrate_spec_err(spec_el_rpsi, spec_el_rpsi_err, g=Relssd)

def calc_Sssdph_from_spec(spec_ph, r, psi, spec_ph_err=None, chi=0):
    Assd = f_Assd(chi)
    Lssd = f_Lssd(chi)
    spec_ph_rpsi = lambda x: spec_ph(r, psi, x)
    if spec_ph_err is not None:
        spec_ph_rpsi_err = lambda x: spec_ph_err(r, psi, x)
    else:
        spec_ph_rpsi_err = None
    return Assd * integrate_spec_err(spec_ph_rpsi, spec_ph_rpsi_err, g=Rphssd)


def calc_Sssdem_from_spec(spec_el, spec_ph, r, psi,
                          spec_el_err=None, spec_ph_err=None, chi=0):
    return ( calc_Sssdel_from_spec(spec_el, r, psi, spec_el_err, chi) + 
            calc_Sssdph_from_spec(spec_ph, r, psi, spec_ph_err, chi) )


def calc_alpha_from_spec(*args):
    Semwcd = calc_Swcdem_from_spec(*args)
    Semssd = calc_Sssdem_from_spec(*args)
    if Semwcd == 0:
        return np.nan
    return Semssd/Semwcd

def calc_beta_from_spec(*args):
    Smuwcd = calc_Swcdmu_from_spec(*args)
    Smussd = calc_Sssdmu_from_spec(*args)
    if Smuwcd == 0:
        return np.nan
    return Smussd/Smuwcd

In [ ]:
fl = '/home/mart/auger/data/UUB_sims/test/Espec_test_DAT111120.root'
# fl = '/home/mart/auger/data/time_templates/Espec_roothists/new/EPOS_LHC/proton/19_19.5/DAT111120_Espec_hists.root'
theta = np.arccos(0.953)
lgE = np.log10(1.54e19)
hist = uproot4.open(fl)

photon_hist, lgr_bins, psi_bins, lgEkin_bins = hist['photon_hist_lgE_19.2_ct_0.95_Xmax_788;1'].to_numpy()
electron_hist, lgr_bins, psi_bins, lgEkin_bins = hist['electron_hist_lgE_19.2_ct_0.95_Xmax_788;1'].to_numpy()
muon_hist, lgr_bins, psi_bins, lgEkin_bins = hist['muon_hist_lgE_19.2_ct_0.95_Xmax_788;1'].to_numpy()

lgrmean = lgr_bins[1:] #(lgr_bins[1:] + lgr_bins[:-1])/2
lgEkin = (lgEkin_bins[1:] + lgEkin_bins[:-1])/2
dlgEkin = (lgEkin_bins[1:] - lgEkin_bins[:-1])

E = 10**lgEkin

psi_mean = (psi_bins[1:] + psi_bins[:-1])/2
psi_mean[0] = -np.pi
psi_mean[-1] = np.pi

# #####################
# ## The area on the ground is an ellipse with major axis r/cos(theta) and minor just r.
# ## Not sure if the psi in the shower plane should change as wel
# ## A_ellipse = pi r^2/cos(theta)
# #####################

# A = Aground_wedge(10**lgr_bins[1:][:, None], 10**lgr_bins[:-1][:, None], 
#                   psi_bins[1:][None, :], psi_bins[:-1][None, :], theta)

A = (np.pi*((10**lgr_bins[1:])**2 - (10**lgr_bins[:-1])**2)/len(psi_mean)/0.95)[:, None]

# #sum over psi
photon_hist = photon_hist/A[:, :, None] / dlgEkin[None, None, :] #/ E /np.log(10)
electron_hist = electron_hist/A[:, :, None] / dlgEkin[None, None,:] #/ E /np.log(10)
muon_hist = muon_hist/A[:, :, None] / dlgEkin[None, None, :] #/ E/np.log(10)
fin_muon = RegularGridInterpolator((lgrmean, psi_mean, lgEkin), muon_hist, bounds_error=False,
                                  fill_value=0)
fin_electron = RegularGridInterpolator((lgrmean, psi_mean, lgEkin), electron_hist, bounds_error=False,
                                  fill_value=0)
fin_photon = RegularGridInterpolator((lgrmean, psi_mean, lgEkin), photon_hist, bounds_error=False,
                                  fill_value=0)

spec_mu_ex = lambda r, psi, lgE: fin_muon((np.log10(r), psi, lgE))
spec_el_ex = lambda r, psi, lgE: fin_electron((np.log10(r), psi, lgE))
spec_ph_ex = lambda r, psi, lgE: fin_photon((np.log10(r), psi, lgE))


In [ ]:
##########################
## these should match for r = 1000:
## muonWCDSignal: 30.9067
## muonSSDSignal: 15.1882
## EMWCDSignal: 48.0599
## EMSSDSignal: 66.5608
##
##for r = 600:
## muonWCDSignal: 145.74
## muonSSDSignal: 73.7499
## EMWCDSignal: 331.9
## EMSSDSignal: 439.873
#######################

for r in [1000, 600]:
    Swcdmu = 0
    Sssdmu = 0
    Swcdem = 0
    Sssdem = 0
    n = 20
    for psi in np.linspace(-np.pi, np.pi, n):
        chi = f_chi(r, theta, psi)
        Swcdmu += calc_Swcdmu_from_spec(spec_mu_ex, r, psi, chi=chi)
        Sssdmu += calc_Sssdmu_from_spec(spec_mu_ex, r, psi, chi=chi)
        Swcdem += calc_Swcdem_from_spec(spec_el_ex, spec_ph_ex, r, psi, chi=chi)
        Sssdem += calc_Sssdem_from_spec(spec_el_ex, spec_ph_ex, r, psi, chi=chi)    
        
    print(f"R = {r}")
    
    print("TOY (from spec)")
    print("==========")
    print(f"WCD muon signal: {Swcdmu/n:.1f} VEM")
    print(f"SSD muon signal: {Sssdmu/n:.1f} MIP")
    print(f"WCD em signal: {Swcdem/n:.1f} VEM")
    print(f"SSD em signal: {Sssdem/n:.1f} MIP")


In [ ]:
cuts = {'lgE': (19.15, 19.25), 
        'cosTheta': (0.94, 0.96),
        'Xmax': (780, 790)}

spec_mu, spec_mu_err = get_spec(proton, 'muon', **cuts)
spec_el, spec_el_err = get_spec(proton, 'electron', **cuts)
spec_ph, spec_ph_err = get_spec(proton, 'photon', **cuts)

for r in [1000, 600]:
    Swcdmu = 0
    Sssdmu = 0
    Swcdem = 0
    Sssdem = 0
    n = 20
    for psi in np.linspace(-np.pi, np.pi, n):
        chi = f_chi(r, theta, psi)
        
        Swcdmu += calc_Swcdmu_from_spec(spec_mu, r, psi, chi=chi)
        Sssdmu += calc_Sssdmu_from_spec(spec_mu, r, psi, chi=chi)
        Swcdem += calc_Swcdem_from_spec(spec_el, spec_ph, r, psi, chi=chi)
        Sssdem += calc_Sssdem_from_spec(spec_el, spec_ph, r, psi, chi=chi)    
        
    print(f"R = {r}")
    
    print("TOY (from spec)")
    print("==========")
    print(f"WCD muon signal: {Swcdmu/n:.1f} VEM")
    print(f"SSD muon signal: {Sssdmu/n:.1f} MIP")
    print(f"WCD em signal: {Swcdem/n:.1f} VEM")
    print(f"SSD em signal: {Sssdem/n:.1f} MIP")

In [ ]:
lgE, dlgE, E, dE = get_E(proton)

rs = np.linspace(400, 2000, 20)

f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

psi = 0

for ct in [0.9, 0.5]:
    E_el = []
    E_ph = []

    Nes = []
    Nys = []
    cuts = {'lgE': (19.4, 19.6), 
            'cosTheta': (ct, ct+0.1),
            'Xmax': (750, 900)}
    
    spec_el, spec_el_err = get_spec(proton, 'electron', **cuts)
    spec_ph, spec_ph_err = get_spec(proton, 'photon', **cuts)
    
    for r in rs:
        spec_el_rpsi = lambda x: spec_el(r, psi, x)
        spec_el_err_rpsi = lambda x: spec_el_err(r, psi, x)
        spec_ph_rpsi = lambda x: spec_ph(r, psi, x)
        spec_ph_err_rpsi = lambda x: spec_ph_err(r, psi, x)
        Ne = integrate_spec_err(spec_el_rpsi, spec_el_err_rpsi)
        E_el.append(integrate_spec_err(spec_el_rpsi, 
                                       spec_el_err_rpsi,
                                       lambda x: 10**x, norm=True)/1e6)
        Ny = integrate_spec_err(spec_ph_rpsi, spec_ph_err_rpsi)
        E_ph.append(integrate_spec_err(spec_ph_rpsi, 
                                       spec_ph_err_rpsi,
                                       lambda x: 10**x, norm=True)/1e6)
        Nes.append(Ne)
        Nys.append(Ny)

        
    tmin = np.rad2deg(np.arccos(ct+0.1))
    tmax = np.rad2deg(np.arccos(ct))
    
    E_el = np.array(E_el)
    E_ph = np.array(E_ph)

    Ny = unumpy.nominal_values(Nys)
    Ne = unumpy.nominal_values(Nes)
    Ny_err = unumpy.std_devs(Nys)
    Ne_err = unumpy.std_devs(Nes)

    ry = Ny/Ne

    ry_err = ry*np.sqrt((Ny_err/Ny)**2 + (Ne_err/Ne)**2 - 2 * np.corrcoef(Ny, Ne)[0, 1]*Ne_err*Ny_err/(Ny*Ne))
    
    ax1.errorbar(rs, unumpy.nominal_values(E_el),
             label=f'${tmin:.0f} < \\theta/^\circ < {tmax:.0f}$')
    ax2.errorbar(rs, unumpy.nominal_values(E_ph))#, yerr=unumpy.std_devs(E_ph), ls='', marker='o')
    ax3.errorbar(rs, ry, ls='-')
#     ax3.errorbar(rs, unumpy.nominal_values(Nes), yerr=unumpy.std_devs(Nes), ls='', marker='v')
#     ax3.errorbar(rs, unumpy.nominal_values(Nys), yerr=unumpy.std_devs(Nys), ls='', marker='x')
#     ax3.set_yscale('log')
ax1.legend(fontsize=14, loc=2)
ax1.set_ylabel('$\\langle E^e \\rangle$ [MeV]')
ax2.set_ylabel('$\\langle E^\gamma \\rangle$ [MeV]')
ax3.set_ylabel('$r^\gamma$')
plt.tight_layout()
ax1.set_xlabel('$r$ [m]')
ax2.set_xlabel('$r$ [m]')
ax3.set_xlabel('$r$ [m]')
plt.tight_layout()
plt.savefig('Ee_Eph_ry_vs_r.pdf', bbox_inches='tight')

In [ ]:


f, ax = plt.subplots(1, figsize=(8, 6))

h5 = proton
lgE, dlgE, E, dE = get_E(h5)


cuts = {'lgE': (18.9, 19.1),
        'cosTheta': (0.95, 1.1),
        'Xmax': (0, 1500)}

spec_el, spec_el_err = get_spec(h5, 'electron', **cuts)
spec_ph, spec_ph_err = get_spec(h5, 'photon', **cuts)
spec_mu, spec_mu_err = get_spec(h5, 'muon', **cuts)
lgE = np.linspace(5, 12)

r = 800
psi = 0

ax.plot(10**lgE, spec_el(r, psi, lgE), color='r', label='$e$')
ax.plot(10**lgE, spec_ph(r, psi, lgE), color='y', label='$\\gamma$')
ax.plot(10**lgE, spec_mu(r, psi, lgE), color='b', label='$\\mu$')

ax.set_yscale('log')
ax.set_xscale('log')

alpha = calc_alpha_from_spec(spec_el, spec_ph, r, psi, spec_el_err, spec_ph_err)
beta = calc_beta_from_spec(spec_mu, r, psi, spec_mu_err)

print(alpha, beta)

ax.set_xlabel('$E_{\\rm kin}$ [eV]')
ax.set_ylabel('$\\frac{d\\rho}{d\\lg{\\left(E/\\mathrm{eV}\\right)}}\ [\\mathrm{m^{-2}}]$')
ax.set_yscale('log')  
ax.legend(fontsize=16)
ax.set_xscale('log')
ax.set_xlim([5e5, 1e11])
plt.tight_layout()
plt.savefig('energy_spec_r800_lgE19_0deg.pdf', bbox_inches='tight')

In [ ]:
r_bins = proton.root.bins.r_bins.read()
npsi = len(proton.root.bins.psi_bins) - 1

A = np.pi*(r_bins[1:]**2 - r_bins[:-1]**2)/npsi

cosTheta = 0.90
lgEcr = 19.5

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
lss = ['-', '--', ':']
for i, r in enumerate([600, 1200, 1800]):

    psi = 0

    h5 = proton
    lgE = np.linspace(5, 12)

    cuts = {'lgE': (lgEcr, lgEcr+0.5), 
            'cosTheta': (cosTheta, cosTheta+0.1),
            'Xmax': (0, 1500)}

    spec_el_, _ = get_spec(h5, 'electron', **cuts)
    spec_ph_, _ = get_spec(h5, 'photon', **cuts)
    spec_mu_, _ = get_spec(h5, 'muon', **cuts)
    spec_el = lambda x: spec_el_(r, psi, x)
    spec_ph = lambda x: spec_ph_(r, psi, x)
    spec_mu = lambda x: spec_mu_(r, psi, x)
    
    Ne = integrate_spec(spec_el)
    Nph = integrate_spec(spec_ph)
    Nmu = integrate_spec(spec_mu)
    ax1.plot(10**lgE, spec_el(lgE)/Ne, 'r', ls=lss[i])
    ax1.plot([0, 0], 'k', ls=lss[i],
             label=f'$r = {r}\, \mathrm{{m}}$')
    ax1.plot(10**lgE, spec_ph(lgE)/Nph, 'y', ls=lss[i])
    ax1.plot(10**lgE, spec_mu(lgE)/Nmu, 'b', ls=lss[i])

ct_bins = np.linspace(1, 0.5, 4)
    
for i, ct in enumerate(ct_bins[:-1]):

    r = 800
    psi = 0
    h5 = proton
    lgE = np.linspace(5, 12)

    cuts = {'lgE': (lgEcr, lgEcr+0.1), 
            'cosTheta': (ct_bins[i+1], ct),
            'Xmax': (0, 1500)}

    spec_el_, _ = get_spec(h5, 'electron', **cuts)
    spec_ph_, _ = get_spec(h5, 'photon', **cuts)
    spec_mu_, _ = get_spec(h5, 'muon', **cuts)
    spec_el = lambda x: spec_el_(r, psi, x)
    spec_ph = lambda x: spec_ph_(r, psi, x)
    spec_mu = lambda x: spec_mu_(r, psi, x)
    
    Ne = integrate_spec(spec_el)
    Nph = integrate_spec(spec_ph)
    Nmu = integrate_spec(spec_mu)

    ax2.plot(10**lgE, spec_el(lgE)/Ne, 'r', ls=lss[i])
    ax2.plot([0, 0], 'k', ls=lss[i], label='$%.0f < \\theta/^\circ < %.0f$' % (np.rad2deg(np.arccos(ct)), 
                                                       np.rad2deg(np.arccos(ct_bins[i+1]))))
    ax2.plot(10**lgE, spec_ph(lgE)/Nph, 'y', ls=lss[i])
    ax2.plot(10**lgE, spec_mu(lgE)/Nmu, 'b', ls=lss[i])  
    
for ax in [ax1, ax2]:
    
    ax.set_xlabel('$E_{\\rm kin}$ [eV]')
    ax.set_ylabel('$\\frac{d\\rho}{d\\lg{\\left(E/\\mathrm{eV}\\right)}}\ [\\mathrm{m^{-2}}]$')#     ax.set_yscale('log')
    ax.set_xscale('log')
    ax.legend()
    ax.set_xlim([1e5, 1e12])

    
plt.tight_layout()

plt.savefig('normalized_spec.pdf', bbox_inches='tight')

Notice that the muon spectrum is cutoff at 100 MeV and this might be a problem, although it is supposed to drop of rapidly. Also notice that the EM spectra are dominated by the low end.

In [ ]:
from scipy.optimize import curve_fit

def bootstrap_curve_fit(func, x, y, yerr=None, p0=None, bounds=(-np.inf, np.inf), nboot=200):
    """ Assuming gaussian errors"""
    x = np.array(x)
    y = np.array(y)
    mask = np.isfinite(y)
    x = x[mask]
    y = y[mask]
    if yerr is not None:
        yerr = np.array(yerr)
    ps = []

    p, pcov = curve_fit(func, x, y, sigma=yerr,
                           absolute_sigma=True, maxfev=2000, p0=p0, bounds=bounds)
    perr = np.sqrt(np.diag(pcov))
    ps.append(p)
    for i in range(nboot):
        if nboot > 1:
            if yerr is None:
                delta = np.random.normal(0, np.std(y-func(x, *p)), size=len(y))
            else:
                delta = np.random.normal(0, yerr, size=len(yerr))
        else:
            delta = 0
        popt, pcov = curve_fit(func, x, y+delta, sigma=yerr,
                               absolute_sigma=True, maxfev=2000, p0=p0, bounds=bounds)
        ps.append(popt)

    ps = np.array(ps)
    p = np.mean(ps, axis=0)
    perr = 0.5*(np.quantile(ps, 0.84, axis=0) - np.quantile(ps, 0.16, axis=0))
    if yerr is None:
        yerr = 1
    chi2 = np.sum(((y-func(x, *p))/yerr)**2)
    ndof = len(x) - len(p)

    return p, perr, chi2/ndof



### $\alpha$ and $\beta$ from spectrum
With the detector response and efficiencies from above
NOTE: The dependence on the track-length is ignored here because it is not readily available and depends on theta and r and psi

From Kamata & Nishimura: "[...] the lateral energy distribution of energy flow is 1/r times steeper than the particle number. This is due to the simple fact that the particles with high energies are less scattered from the shower axis [...] At a large distance, however, ge and gy increase nearly in proportion
to the distance from the shower axis, thus the average energy of the
shower particles remains almost constant in this region. The result may
be interpreted in the following way. The mean free path for the pair
creation of the photons remains almost constant for all over the energies.
Thus low energy photons can travel up to a large distance from the shower
axis, and the electrons lying in this region are produced from these low
energy photons. These low energy electrons can travel only a short distance
because they lose their energies by the ionization process. As a result the
average energies of the shower particles at varying distances from the axis
remain constant, and photons are much more accumulated at a large distance
from the core compared with the electrons".

\begin{align}
\alpha(r) = a \left(\frac{r}{1000}\right)^b \left(1+\frac{r}{1000}\right)^c
\end{align}


The muon spectrum depends on the path length of the muons traveled from their production (see L Cazon etc.):
\begin{align}
l = \sqrt{r^2 + z^2}
\end{align}
and 
\begin{align}
\frac{d\rho^\mu}{dE} \sim l^c
\end{align} (something like this, but very roughly this only holds at low energy end)
so then:
\begin{align}
\beta(r) = a \left[ \left(\frac{r}{1000}\right)^2 + b^2 \sec^2{\theta} \right]^{c/2}
\end{align}

In [ ]:
cuts={'lgE': (19.0, 19.5), 'cosTheta': (0.5, 1), 'Xmax': (750, 900)}


f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7), sharex=True)   
rs = np.linspace(400, 2000, 20)



for i, ct in enumerate([0.6, 0.9]):
    cuts['cosTheta'] = (ct, ct+0.1)
    spec_el, spec_el_err = get_spec(h5, 'electron', **cuts)
    spec_ph, spec_ph_err = get_spec(h5, 'photon', **cuts)
    spec_mu, spec_mu_err = get_spec(h5, 'muon', **cuts)
    tmin = np.rad2deg(np.arccos(ct+0.1))
    tmax = np.rad2deg(np.arccos(ct))
    for j, psi in enumerate([0, np.pi]):
        alphas = []
        betas = []
        for r in rs:
            alphas.append(calc_alpha_from_spec(spec_el, spec_ph, r, psi))
            betas.append(calc_beta_from_spec(spec_mu, r, psi))
            
            
        label = f'${tmin:.0f} < \\theta/^\circ < {tmax:.0f}$, $\\psi = {np.rad2deg(psi):.0f}^\circ$'
        ax1.errorbar(rs, alphas,
                     color=colors[i], marker=markers[j+1], ls='', label=label)
        ax2.errorbar(rs, betas,
                     color=colors[i], marker=markers[j+1], ls='')
        
        p, perr, chi2r = bootstrap_curve_fit(f_r_alpha, rs, alphas, nboot=0, 
                                 p0=[1.4, 0.5, 2],
                                 bounds=[(0, -1, -1), (10, 2, 3)])
        ax1.plot(rs, f_r_alpha(rs, *p), color=colors[i], ls=lss[j])
        
        p, perr, chi2r = bootstrap_curve_fit(f_r_beta, rs, betas, nboot=0, 
                                        p0=[2, 0.5, 0.3], bounds=[(0, 0, -1), (10, 2, 2)])
        ax2.plot(rs, f_r_beta(rs, *p), color=colors[i], ls=lss[j])
        
ax1.legend(fontsize=14)
# ax1.set_ylim([0.5, 1.9])
ax1.set_ylabel('$\\alpha$ [MIP/VEM]')
ax2.set_ylabel('$\\beta$ [MIP/VEM]')
plt.tight_layout()
plt.savefig('alpha_beta_toymodel.pdf', bbox_inches='tight')


## $\alpha$ and $\beta$ from reconstructed simulations
<!-- \begin{align}
    \alpha = \frac{S_S^{em}}{S_W^{em} + S_S^{em}} \\
    \beta = \frac{S_S^{\mu}}{S_W^{\mu} + S_S^{\mu}}
\end{align} -->
\begin{align}
    \alpha = \frac{S_S^{em}}{S_W^{em}} \\
    \beta = \frac{S_S^{\mu}}{S_W^{\mu}}
\end{align}

In [ ]:
from time_templates.data.get_data import get_MC_data

def load_data(primary):
    df = get_MC_data(primary=primary, dense=True, no_traces=True)
    df = df.query('LowGainSat == 0 & WCDTotalSignal > 10')
#     df = df.query('LowGainSat == 0')
#     df['alpha'] = df['SSDEMSignal']/(df['SSDEMSignal'] + df['WCDEMSignal'])
#     df['beta'] = df['SSDMuonSignal']/(df['SSDMuonSignal'] + 5df['WCDMuonSignal'])
#     df['alpha'] = df['WCDEMSignal']/df['SSDEMSignal']
#     df['beta'] = df['WCDMuonSignal']/df['SSDMuonSignal']   
#     df['SSDMuonSignal'] *= 175/196
#     df['SSDEMSignal'] *= 175/196
    df['alpha'] = df['SSDEMSignal']/df['WCDEMSignal'] #* 175/196 * 1586/1619
    df['beta'] = df['SSDMuonSignal']/df['WCDMuonSignal']# * 175/196 * 1586/1619
    
    df['MCr'] = df['MCr'].round()
    return df

df_proton = load_data('proton')
# df_iron = load_data('iron')
df_proton.shape

In [ ]:

lgE = 19.8
cosTheta = 0.95
psi = np.pi/2

rs = [600, 800, 1000, 1200, 1400]

f, ax = plt.subplots(1)

wcdmu = []
ssdmu = []
ssdem = []
wcdem = []
for r in rs:
    df_ = df_proton.query(f'MClgE > {lgE-0.02} & MClgE < {lgE+0.02} & \
    MCCosTheta > {cosTheta-0.02} & MCCosTheta < {cosTheta + 0.02} & MCr == {r}')
    wcdmu_true = df_['WCDMuonSignal'].mean()
    ssdmu_true = df_['SSDMuonSignal'].mean()
    wcdem_true = df_['WCDEMSignal'].mean()
    ssdem_true = df_['SSDEMSignal'].mean()
#     print(' "True": ')
#     print(f"WCD Muon signal = {df_['WCDMuonSignal'].mean():.2f} VEM")
#     print(f"SSD Muon signal = {df_['SSDMuonSignal'].mean():.2f} MIP")
#     print(f"WCD EM signal = {df_['WCDEMSignal'].mean():.2f} VEM")
#     print(f"SSD EM signal = {df_['SSDEMSignal'].mean():.2f} MIP")

    cuts = {'lgE': (lgE-0.02, lgE+0.02), 'cosTheta': (cosTheta-0.02, cosTheta+0.02)}

    spec_mu, _ = get_spec(proton, 'muon', **cuts)
    spec_el, _ = get_spec(proton, 'electron', **cuts)
    spec_ph, _ = get_spec(proton, 'photon', **cuts)
    theta = np.arccos(cosTheta)
    psi = np.pi/2
    chi = f_chi(r, theta, psi)
    wcdmu_toy = calc_Swcdmu_from_spec(spec_mu, r, psi, chi=chi)
    ssdmu_toy = calc_Sssdmu_from_spec(spec_mu, r, psi, chi=chi)
    wcdem_toy = calc_Swcdem_from_spec(spec_el, spec_ph, r, psi, chi=chi)
    ssdem_toy = calc_Sssdem_from_spec(spec_el, spec_ph, r, psi, chi=chi)

    wcdmu.append(wcdmu_true/wcdmu_toy)
    ssdmu.append(ssdmu_true/ssdmu_toy)
    wcdem.append(wcdem_true/wcdem_toy)
    ssdem.append(ssdem_true/ssdem_toy)

#     print("Toy:")
#     print(f"WCD Muon signal = {calc_Swcdmu_from_spec(spec_mu, r, psi):.2f} VEM")
#     print(f"SSD Muon signal = {calc_Sssdmu_from_spec(spec_mu, r, psi):.2f} MIP")
#     print(f"WCD EM signal = {calc_Swcdem_from_spec(spec_el, spec_ph, r, psi):.2f} VEM")
#     print(f"SSD EM signal = {calc_Sssdem_from_spec(spec_el, spec_ph, r, psi):.2f} MIP")

ax.plot(rs, wcdmu, 'bo')
ax.plot(rs, ssdmu, 'bv')
ax.plot(rs, ssdem, 'rv')
ax.plot(rs, wcdem, 'ro')
ax.axhline(1, ls='--', color='k')
ax.set_ylabel('Strue/Stoy')
ax.set_xlabel('r [m]')
ax.set_title(f'cos(theta) = {cosTheta:.2f}')

In [ ]:

lgE = 19.8
cosTheta = 0.95
psi = np.pi/2

cts = np.linspace(0.55, 0.95, 5)

f, ax = plt.subplots(1)

r = 600
wcdmu = []
ssdmu = []
ssdem = []
wcdem = []

for ct in cts:
    df_ = df_proton.query(f'MClgE > {lgE-0.02} & MClgE < {lgE+0.02} & \
    MCCosTheta > {ct-0.02} & MCCosTheta < {ct + 0.02} & MCr == {r}')
    wcdmu_true = df_['WCDMuonSignal'].mean()
    ssdmu_true = df_['SSDMuonSignal'].mean()
    wcdem_true = df_['WCDEMSignal'].mean()
    ssdem_true = df_['SSDEMSignal'].mean()
#     print(' "True": ')
#     print(f"WCD Muon signal = {df_['WCDMuonSignal'].mean():.2f} VEM")
#     print(f"SSD Muon signal = {df_['SSDMuonSignal'].mean():.2f} MIP")
#     print(f"WCD EM signal = {df_['WCDEMSignal'].mean():.2f} VEM")
#     print(f"SSD EM signal = {df_['SSDEMSignal'].mean():.2f} MIP")

    cuts = {'lgE': (lgE-0.02, lgE+0.01), 'cosTheta': (cosTheta-0.02, cosTheta+0.02)}

    spec_mu, _ = get_spec(proton, 'muon', **cuts)
    spec_el, _ = get_spec(proton, 'electron', **cuts)
    spec_ph, _ = get_spec(proton, 'photon', **cuts)
    theta = np.arccos(ct)
    chi = f_chi(r, theta, psi)
    wcdmu_toy = calc_Swcdmu_from_spec(spec_mu, r, psi, chi=chi)
    ssdmu_toy = calc_Sssdmu_from_spec(spec_mu, r, psi, chi=chi)
    wcdem_toy = calc_Swcdem_from_spec(spec_el, spec_ph, r, psi, chi=chi)
    ssdem_toy = calc_Sssdem_from_spec(spec_el, spec_ph, r, psi, chi=chi)

    wcdmu.append(wcdmu_true/wcdmu_toy)
    ssdmu.append(ssdmu_true/ssdmu_toy)
    wcdem.append(wcdem_true/wcdem_toy)
    ssdem.append(ssdem_true/ssdem_toy)

#     print("Toy:")
#     print(f"WCD Muon signal = {calc_Swcdmu_from_spec(spec_mu, r, psi):.2f} VEM")
#     print(f"SSD Muon signal = {calc_Sssdmu_from_spec(spec_mu, r, psi):.2f} MIP")
#     print(f"WCD EM signal = {calc_Swcdem_from_spec(spec_el, spec_ph, r, psi):.2f} VEM")
#     print(f"SSD EM signal = {calc_Sssdem_from_spec(spec_el, spec_ph, r, psi):.2f} MIP")

ax.plot(cts, wcdmu, 'bo')
ax.plot(cts, ssdmu, 'bv')
ax.plot(cts, ssdem, 'rv')
ax.plot(cts, wcdem, 'ro')
ax.axhline(1, ls='--', color='k')
ax.set_ylabel('Strue/Stoy')
ax.set_xlabel('cos(theta)')
ax.set_title(f'r = {r:.2f} m')


In [ ]:
f, axes = plt.subplots(4, 2, figsize=(10, 12), sharex='col', sharey=False)

from time_templates.utilities.stats import get_kde_mode
from scipy.stats import gaussian_kde

key = 'beta'

for i, r in enumerate([600, 1000, 1400, 1800]):
    df_p = df_proton.query(f'(MCr == @r) & (MCCosTheta < 0.6) & (alpha < 10)')

    ax1 = axes[i, 0]
    ax1.hist(df_p['alpha'], 
            bins=np.linspace(0., 2.5, 25), label='r = %.0f m'%r, density=True)
    ax1.axvline(df_p['alpha'].mean(), ls='-', color='k', label='mean')
    ax1.axvline(np.median(df_p['alpha']), ls=':', color='k', label='median')
    try:
        kde = gaussian_kde(df_p['alpha'])
        x = np.linspace(0., 2.5, 1000)
        ax1.plot(x, kde.pdf(x), 'r--')
    except:
        pass
    ax1.legend(fontsize=12, loc=1)
    df_p = df_proton.query(f'(MCr == @r) & (MCCosTheta < 0.6) & (beta < 10)')

    ax2 = axes[i, 1]
    ax2.hist(df_p['beta'], 
            bins=np.linspace(0, 2, 25), label='r = %.0f m'%r, density=True)
    ax2.axvline(df_p['beta'].mean(), ls='-', color='k', label='mean')
    ax2.axvline(np.median(df_p['beta']), ls=':', color='k', label='median')
    try:
        kde = gaussian_kde(df_p['beta'])
        x = np.linspace(0, 2, 1000)
        ax2.plot(x, kde.pdf(x), 'r--')
    except:
        pass
    
ax1.set_xlabel('$\\alpha$')
ax2.set_xlabel('$\\beta$')
# ax.set_xlim([0, 1])
plt.tight_layout()
plt.subplots_adjust(hspace=0.07)

In [ ]:
# sct_bins = np.linspace(1, 2, 6)
ct_bins = np.linspace(0.5, 1, 6)
mct = (ct_bins[:-1] + ct_bins[1:])/2

psi_bins = [0,  np.pi/2, np.pi]

nct = len(ct_bins) - 1
npsi = len(psi_bins) - 1

f, axes = plt.subplots(nct, 2, figsize=(14, 18), sharex=True, sharey='col')   

beta_p2 = np.zeros((nct, npsi, 3))
beta_perr2 = np.zeros((nct, npsi, 3))

alpha_p2 = np.zeros((nct, npsi, 3))
alpha_perr2 = np.zeros((nct, npsi, 3))
    
markers = ['o', 'v', 'x', 's', '^']
# lss = ['-', '--', ':', '-.', '']
r_bins = np.arange(300, 2200, 200)
r_bins_cut = r_bins[6:]
r_bins = r_bins[:7]
rspace = np.linspace(300, 3000)

stat = 'mode'

outliers = (-100, 100)

for i in range(nct):
    print("Working on i = ", i, end=' ')
    ctl = ct_bins[i]
    cth = ct_bins[i+1]
    ax1 = axes[i, 0]
    ax2 = axes[i, 1]
    for j in range(npsi):
        psil = psi_bins[j]
        psih = psi_bins[j+1]
 
        df_ = df_proton.query('MCCosTheta >= @ctl & MCCosTheta < @cth & MCpsi >= @psil & MCpsi <= @psih')
        
        _, (x, y, yerr) = plot_profile_1d(df_['MCr'], df_['alpha'], bins=r_bins, stat=stat,
                       ax=ax1, bootstraps=0, remove_outliers=outliers, marker=markers[i],
                                         color=colors[j],
                                         label=f'${np.rad2deg(psil):.0f} < \\psi/^\circ < {np.rad2deg(psih):.0f}$')
        _, _ = plot_profile_1d(df_['MCr'], df_['alpha'], bins=r_bins_cut, stat=stat,
                       ax=ax1, marker=markers[i], color=colors[j], mfc='none', remove_outliers=outliers)
                                         
        sct = (sct_bins[i] + sct_bins[i+1])/2

#         f_r_alpha = lambda r, a, b, c: a*(r/700)**b*(1+(r/700))**-c
        p, perr, chi2r = bootstrap_curve_fit(f_r_alpha, x, y, yerr, nboot=0,
                                           p0=[1.4, 0.5, 2],
                                             bounds=[(0, -1, -1), (10, 2, 3)])
        alpha_p2[i, j] = p
        alpha_perr2[i, j] = perr
        ax1.plot(rspace, f_r_alpha(rspace, *p), color=colors[j])


        _, (x, y, yerr) = plot_profile_1d(df_['MCr'], df_['beta'], bins=r_bins, stat=stat,
                       ax=ax2, bootstraps=0, remove_outliers=outliers, marker=markers[i],
                                          color=colors[j])
        _, _ = plot_profile_1d(df_['MCr'], df_['beta'], bins=r_bins_cut, stat=stat,
                       ax=ax2, marker=markers[i], color=colors[j], mfc='none', remove_outliers=outliers)  
        
#         f_r_beta = lambda r, a, b, c: a * np.sqrt((r/700)**2 + (b*sct)**2)**c
 
        p, perr, chi2r = bootstrap_curve_fit(f_r_beta, x, y, yerr, nboot=0,
                                        p0=[2, 0.5, 0.3], bounds=[(0, 0, -1), (10, 2, 2)])
        beta_p2[i, j] = p
        beta_perr2[i, j] = perr
        ax2.plot(rspace, f_r_beta(rspace, *p), color=colors[j])

        ax1.set_ylabel('$\\alpha$')
        ax2.set_ylabel('$\\beta$')
        ax1.set_title(f'${ctl:.2f} < \\cos{{\\theta}} < {cth:.2f}$')
        ax2.set_title(f'${ctl:.2f} < \\cos{{\\theta}} < {cth:.2f}$')
        ax2.set_xlabel('$r$ [m]')
        ax1.set_xlabel('$r$ [m]')
        ax1.set_ylim([0.9, 1.6])
        ax2.set_ylim([0.3, 0.7])
axes[0, 0].legend()


plt.tight_layout()
# plt.savefig('alpha_beta_from_signal_Swcd20.pdf', bbox_inches='tight')

In [ ]:
# df = get_MC_data(grouped=True)
df = get_MC_data(dense=True)
df = df.query('LowGainSat == 0 & MClgE >= 19.5 & MClgE < 19.6 & MCpsi > 1 & MCpsi < 1.4')

In [ ]:
# df = df.query('LowGainSat == 0 & WCDTotalSignal > 15 & MClgE > 19')
df['alpha'] = df['SSDEMSignal']/df['WCDEMSignal'] #* 175/196 * 1586/1619
df['beta'] = df['SSDMuonSignal']/df['WCDMuonSignal']# * 175/196 * 1586/1619
df['MCr'] = df['MCr'].round()

In [ ]:
r_bins

In [ ]:
# sct_bins = np.linspace(1, 2, 6)
ct_bins = np.linspace(0.6, 1, 7)
mct = (ct_bins[:-1] + ct_bins[1:])/2
nct = len(ct_bins) - 1

f, axes = plt.subplots(nct, 2, figsize=(14, 18), sharex=True, sharey='col')   

beta_p2 = np.zeros((nct, 2))
beta_perr2 = np.zeros((nct, 2))

alpha_p2 = np.zeros((nct, 2))
alpha_perr2 = np.zeros((nct, 2))

r_bins = np.arange(500, 2000, 200)
# r_bins_cut = r_bins[6:]
# r_bins = r_bins[:7]
rspace = np.linspace(300, 3000)

stat = 'mean'
outliers = (0, 10)
nboot = 100
for i in range(nct):
    print("Working on i = ", i, end=' ')
    ctl = ct_bins[i]
    cth = ct_bins[i+1]
    ax1 = axes[i, 0]
    ax2 = axes[i, 1]

    df_ = df.query('MCCosTheta >= @ctl & MCCosTheta < @cth')
#     outliers = (np.quantile(df_['alpha'], 0.01), np.quantile(df_['alpha'], 0.99))
    _, (x, y, yerr) = plot_profile_1d(df_['MCr'], df_['alpha'], bins=r_bins, stat=stat,
                   ax=ax1, bootstraps=nboot, remove_outliers=outliers, 
                                     color=colors[0])

    _, _ = plot_profile_1d(df_['MCr'], df_['alpha'], bins=r_bins_cut, stat=stat,
                   ax=ax1,color=colors[0], mfc='none', remove_outliers=outliers)
    
    
    f_r_alpha = lambda r, a, b: a*(r/700) * (1+(r/700))**-b
    f_r_beta = lambda r, a, b, c: a*((r/700)**2 + c**2)**b
    
    p, perr, chi2r = bootstrap_curve_fit(f_r_alpha, x, y, yerr, nboot=nboot)
#     alpha_p2[i] = p
#     alpha_perr2[i] = perr
    ax1.plot(rspace, f_r_alpha(rspace, *p), color=colors[0])

#     outliers = (np.quantile(df_['beta'], 0.01), np.quantile(df_['beta'], 0.99))
#     print(outliers)
    _, (x, y, yerr) = plot_profile_1d(df_['MCr'], df_['beta'], bins=r_bins, stat=stat,
                   ax=ax2, bootstraps=nboot, remove_outliers=outliers, 
                                      color=colors[0])
    _, _ = plot_profile_1d(df_['MCr'], df_['beta'], bins=r_bins_cut, stat=stat,
                   ax=ax2, color=colors[0], mfc='none', remove_outliers=outliers)  


    p, perr, chi2r = bootstrap_curve_fit(f_r_beta, x, y, yerr, nboot=nboot,
                                    p0=[0.5, 0.1, 0],
                                        bounds=[(0, -1, 0), (10, 1, 10)])
#     beta_p2[i] = p
#     beta_perr2[i] = perr
    ct = (ctl+cth)/2
    ax2.plot(rspace, f_r_beta(rspace, *p), color=colors[0])

    ax1.set_ylabel('$\\alpha$')
    ax2.set_ylabel('$\\beta$')
    ax1.set_title(f'${ctl:.2f} < \\cos{{\\theta}} < {cth:.2f}$')
    ax2.set_title(f'${ctl:.2f} < \\cos{{\\theta}} < {cth:.2f}$')
    ax2.set_xlabel('$r$ [m]')
    ax1.set_xlabel('$r$ [m]')
    ax1.set_ylim([0.9, 2])
    ax2.set_ylim([0.4, 1])


plt.tight_layout()
# plt.savefig('alpha_beta_from_signal_Swcd20.pdf', bbox_inches='tight')

In [ ]:
f, axes = plt.subplots(2, 2, figsize=(15, 10))

for i in range(2):
    axes[0, i].errorbar(mct, alpha_p2[:, i], yerr=alpha_perr2[:, i], marker='o', ls='', color='C3')
    
    if i == 2:
        continue
    axes[1, i].errorbar(mct, beta_p2[:, i], yerr=beta_perr2[:, i], marker='o', color='C0', ls='')
    p, perr, chi2 = bootstrap_curve_fit(lambda x, a, b: a+b*x, 
                                        mct, beta_p2[:, i], yerr=beta_perr2[:, i], nboot=200)
    axes[1, i].plot(mct, p[0]+p[1]*mct, 'C0--')
    print(p)

In [ ]:
f_alpha = lambda r: 6.75*(r/700) * (1+(r/700))**-2.08
f_beta = lambda r, cosTheta: (0.233+0.5*cosTheta)*(r/700)**(0.05+0.07*cosTheta)
df['alpha_pred'] = f_alpha(df['MCr'])
df['beta_pred'] = f_beta(df['MCr'], df['MCCosTheta'])

In [ ]:
ax, _ = plot_profile_1d(df['Sdr'], df['alpha_pred']/df['alpha'] - 1, bins=np.arange(300, 2200, 200),
                       remove_outliers=(-1, 1));
plot_profile_1d(df['Sdr'], df['beta_pred']/df['beta'] - 1, bins=np.arange(300, 2200, 200),
                       remove_outliers=(-1, 1), ax=ax);
ax.axhline(0, ls='--', color='k')

In [ ]:
ax, _ = plot_profile_1d(df['SdCosTheta'], df['alpha_pred']/df['alpha'] - 1, bins=np.linspace(0.5, 1, 9),
                       remove_outliers=(-1, 1));
plot_profile_1d(df['SdCosTheta'], df['beta_pred']/df['beta'] - 1, bins=np.linspace(0.5, 1, 9),
                       remove_outliers=(-1, 1), ax=ax);
ax.axhline(0, ls='--', color='k')

In [ ]:
df = get_MC_data(dense=True)
df = df.query('LowGainSat == 0 & MClgE >= 19.5 & MClgE < 19.6 & MCpsi > 1 & MCpsi < 1.4')

In [ ]:
def ldf(r, s, rx=700):
    return (r/rx)**(s-2) * (1+r/rx)**(s-9/2)

def LDF(r, S1000, s, rx=700):
    C = S1000/ldf(1000, s, rx)
    return C * ldf(r, s, rx)

In [ ]:
np.arange(300, 2200, 200)

In [ ]:
nct = 6

f, axes = plt.subplots(nct, 4, figsize=(15, 20))

ct_bins = np.linspace(0.6, 1, nct+1)
# r_bins = np.sort(df['MCr'].round().unique())[:-1]+100
r_bins = np.arange(300, 2200, 200)
rspace = np.linspace(300, 2500)

S1000_p = np.zeros((nct, 4))
S1000_perr = np.zeros((nct, 4))

alpha_p = np.zeros((nct, 4))
beta_p = np.zeros((nct, 4))

alpha_perr = np.zeros((nct, 4))
beta_perr = np.zeros((nct, 4))

keys = ['WCDMuonSignal', 'WCDEMSignal', 'SSDMuonSignal', 'SSDEMSignal']
nboot = 0

for i in range(nct):
    print(f'\r {i+1}/{nct}', end='')
    ctl = ct_bins[i]
    cth = ct_bins[i+1]
    df_ = df.query('MCCosTheta > @ctl & MCCosTheta <= @cth')
    for j, key in enumerate(keys):
        ax = axes[i, j]
        ax, (x, y, yerr) = plot_profile_1d(df_['MCr'], df_[key], bins=r_bins, ax=ax, bootstraps=0,
                                          remove_outliers=(0, 1000), stat='mean')
        mask = np.isfinite(x)&np.isfinite(y)&np.isfinite(yerr) & (yerr > 0)
        p, pcov = curve_fit(LDF, x[mask], y[mask], sigma=yerr[mask], absolute_sigma=False,
                                            p0=[50, 1], bounds=[(0, -6), (1e9, 6)])

        print('')
        print(np.sum(((y[mask]-LDF(x[mask], *p))/yerr[mask])**2), len(x[mask])-2)
        perr = np.sqrt(np.diag(pcov))
        ax.plot(rspace, LDF(rspace, *p))
        ax.set_yscale('log')
        S1000_p[i, j] = p[0]
        S1000_perr[i, j] = perr[0]
        alpha_p[i, j] = p[1]
#         beta_p[i, j] = p[2]
        alpha_perr[i, j] = perr[1]
#         beta_perr[i, j] = perr[2]
plt.tight_layout()

In [ ]:
f, axes = plt.subplots(3, 4, figsize=(15, 10))
mct = (ct_bins[:-1] + ct_bins[1:])/2

for i in range(4):
    ax1 = axes[0, i]
    ax1.errorbar(mct, S1000_p[:, i], yerr=S1000_perr[:, i], marker='o', ls='')
    popt, pcov = curve_fit(lambda x, a, b: a+b/x, mct, S1000_p[:, i], sigma=S1000_perr[:, i])
    ax1.plot(mct, popt[0]+popt[1]/mct, 'r--')
    print(popt)
    ax2 = axes[1, i]
    ax2.errorbar(mct, alpha_p[:, i], yerr=alpha_p[:, i], marker='o', ls='')   
    
    ax3 = axes[2, i]
    ax3.errorbar(mct, beta_p[:, i], yerr=beta_p[:, i], marker='o', ls='')   

In [ ]:
def alpha(r, theta):
    ct = np.cos(theta)
    Sem_w1000 = 222- 113/ct
    Sem_s1000 = 295 - 133/ct
    Sem_w = LDF(r, Sem_w1000, 0.4)
    Sem_s = LDF(r, Sem_s1000, 0.5)
    
    return Sem_s/Sem_w


def beta(r, theta):
    ct = np.cos(theta)
    Smu_w1000 = 55
    Smu_s1000 = 40
    Smu_w = LDF(r, Smu_w1000, 1)
    Smu_s = LDF(r, Smu_s1000, 1.1)
    return Smu_s/Smu_w

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
r = np.linspace(400, 2500)
for theta in np.linspace(0, np.deg2rad(50), 5):
    ax1.plot(r, alpha(r, theta))
    ax2.plot(r, beta(r, theta))

In [ ]:
alpha_1000 = S1000_p[-1, 3]/S1000_p[-1, 1]
beta_1000 = S1000_p[-1, 2]/S1000_p[-1, 0]

print(alpha_1000, beta_1000)